In [ ]:
import config
import gzip
import lib_db
import lib_nlp
import datetime
import lib_vk
import datetime
import pickle

In [ ]:
data = {}

In [ ]:
import json, pickle
datafile = 'saved_data.pkl'
def save_data():
    with open(datafile, 'wb') as f:
        pickle.dump(data, f)
        
def load_data():
    global data
    with open(datafile, 'rb') as f:
        data = pickle.load(f)

In [ ]:
load_data()

In [ ]:
import vk_api
token = ''
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [ ]:
gid = 666  # group_id

In [ ]:
print(lib_vk.get_group_info(gid))

In [ ]:
user_list = lib_vk.get_group_users(gid, iterations=190, debuglines=True, skip_cache=False)

In [ ]:
user_list[200]

In [ ]:
import pprint

In [ ]:
for u in user_list:
    if 'schools' in u:
        pprint.pprint(u)
        break

In [ ]:
from collections import Counter

In [ ]:
cnt_city = Counter()
cnt_university_name = Counter()
cnt_education_status = Counter()
cnt_graduation = Counter()
cnt_faculty_name = Counter()
cnt_sex = Counter()
cnt_age = Counter()

In [ ]:
for u in user_list:
    city = u.get('city', {}).get('title', False)
    if city:
        cnt_city.update([city])
        
    university_name = u.get('university_name', False)
    if university_name:
        cnt_university_name.update([university_name])
        
    education_status = u.get('education_status', False)
    if education_status:
        cnt_education_status.update([education_status])
        
    graduation = u.get('graduation', False)
    if graduation:
        cnt_graduation.update([graduation])
        
    faculty_name = u.get('faculty_name', False)
    if faculty_name:
        cnt_faculty_name.update([faculty_name])
        
    sex = u.get('sex', False)
    if sex:
        cnt_sex.update(['female' if sex == 1 else 'male'])

In [ ]:
cnt_sex.most_common(3)

In [ ]:
cnt_city.most_common(10)

In [ ]:
import pandas as pd
def prepare_df(some_cnt, total_cnt):
    indexes = []
    vals = []
    for v, c in some_cnt.most_common(20):
        indexes.append(v)
        vals.append(int(c)*100/total_cnt)        
    df_data = {}
    df_data['percent'] = pd.Series(vals, index=indexes)
    df = pd.DataFrame(df_data)
    return df

In [ ]:
import pandas as pd
import matplotlib as mpl
# mpl.rc('font', family='Verdana') 
mpl.rc('font', family='DejaVu Sans') 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = prepare_df(cnt_city, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Топ городов')

In [ ]:
df = prepare_df(cnt_sex, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Пол')

In [ ]:
df = prepare_df(cnt_university_name, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Университет')

In [ ]:
df = prepare_df(cnt_education_status, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Образование')

In [ ]:
df = prepare_df(cnt_graduation, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Год выпуска')

In [ ]:
df = prepare_df(cnt_faculty_name, len(user_list))
df.plot(kind='barh', stacked=True, figsize=(6, 5), title='Название факультета')

In [ ]:
from random import shuffle
shuffle(user_list)

In [ ]:
subset_count = 500

In [ ]:
%%time
lib_vk.vk = vk_old
topgroups_cnt = Counter()
for i in range(subset_count):
    u = user_list[i]
    try:
        groups = lib_vk.get_user_groups(u['id'])
    except:
        pass
    topgroups_cnt.update(groups)
    if i % 20 == 0:
        print(i)

In [ ]:
def prepare_groups_df(some_cnt, total_cnt):
    indexes = []
    vals = []
    for v, c in some_cnt.most_common(40):
        if int(v) == int(gid):
            continue
        g_name = lib_vk.get_group_info(v)
        g_name = g_name['name']
        indexes.append(g_name)
        vals.append(int(c)*100/total_cnt)        
    df_data = {}
    df_data['percent'] = pd.Series(vals, index=indexes)
    df = pd.DataFrame(df_data)
    return df

In [ ]:
df = prepare_groups_df(topgroups_cnt, subset_count)
df.plot(kind='barh', stacked=True, figsize=(6, 10), title='Популярные сообщества')

In [ ]:
with open('all_users.pkl', 'rb') as f:
    all_users = pickle.load(f)